<a href="https://colab.research.google.com/github/hsym-uec/myWeb_github/blob/main/Simmple_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# SoftBank Stock data from Stooq.com 
# reference https://seanmemo.com/234/

from pandas_datareader import data
start = '2023-04-01'
end = '2023-04-19'
df = data.DataReader('9984.JP', 'stooq', start, end)


ImportError: cannot import name 'get_host' from 'urllib3.util.url' (/Users/natoririku/Documents/GitHub/python-scraping-stock-data-UEC-Natori/jupyter_venv/lib/python3.9/site-packages/urllib3/util/url.py)

In [14]:
from pandas_datareader import data
import pandas as pd
import datetime as dt
from dateutil import relativedelta


#今日の日付を取得します。
dateNow = dt.datetime.today()

#1年前を取得
dateYearBefore = dateNow - relativedelta.relativedelta(years=1)

#yahooから1年前から今日までのS&P500データを取得する
df = data.DataReader("^GSPC" ,"yahoo", start=dateYearBefore, end=dateNow)

ImportError: cannot import name 'get_host' from 'urllib3.util.url' (/Users/natoririku/Documents/GitHub/python-scraping-stock-data-UEC-Natori/jupyter_venv/lib/python3.9/site-packages/urllib3/util/url.py)

In [8]:
df

""
Exceeded the daily hits limit


In [7]:
import pandas as pd

def get_price(code):
    df = pd.read_csv(f'https://stooq.com/q/d/l/?s={code}&i=d',index_col=0)
    df.index = pd.to_datetime(df.index).tz_localize('Asia/Tokyo')
    return df

df = get_price('6701.jp')

In [ ]:
### update balance

import mysql.connector
import datetime

def insert_balance(balance):

    cnx = None
    dt_now = datetime.datetime.now()

    try:
        cnx = mysql.connector.connect(
            user='root',  # ユーザー名
            password='',  # パスワード
            host='localhost',  # ホスト名(IPアドレス）
            database='stock_test'  # データベース名
        )

        cursor = cnx.cursor()

        sql = ('''
        INSERT INTO wallent02 
            (balance, datetime)
        VALUES 
            (%s, %s)
        ''')

        data = [
            (balance, dt_now)
        ]

        cursor.executemany(sql, data)
        cnx.commit()

        print(f"{cursor.rowcount} records inserted.")

        cursor.close()

    except Exception as e:
        print(f"Error Occurred: {e}")

    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()


insert_balance(-1000)


1 records inserted.


In [ ]:
!pip3 install mysql-connector-python

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 8.4 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 162 kB 1.2 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
### get latest balance

import mysql.connector

def get_balance():

    cnx = None
    current_balance = 0

    try:
        cnx = mysql.connector.connect(
            user='root',  # ユーザー名
            password='',  # パスワード
            host='localhost',  # ホスト名(IPアドレス）
            database='stock_test'  # データベース名
        )

        cursor = cnx.cursor()

        sql = ('''
        SELECT balance
        FROM wallent02
        ''')

        cursor.execute(sql)
        #cnx.commit()

        print(f"Last balance gotten.")

        for row in cursor.fetchall():
          current_balance += row[0]

        cursor.close()

    except Exception as e:
        print(f"Error Occurred: {e}")

    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()
        return current_balance


get_balance()

Last balance gotten.


60400.0

In [12]:
!pip install --upgrade pip
!pip install pandas-datareader
!pip install urllib3==1.26.6

     |████████████████████████████████| 2.1 MB 6.9 MB/s eta 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 21.2.4
    Uninstalling pip-21.2.4:
      Successfully uninstalled pip-21.2.4
